## The LSTM

The Long Short-Term Memory network, or LSTM network, is a recurrent neural network that is trained using Backpropagation Through Time and overcomes the vanishing gradient problem.

With RNNs, the real substance of the model were the hidden neurons. These units did the processing on the input through time time to produce the outputs. At each timestep, hidden neuron embodies a hidden state that is computed by feeding the weighted sum vetor of the input or previous hidden states with an added bias vector through a tanh squashing function.

In comparison to LSTMs, hidden states still exist, but they are calculated through a LSTM cell. Instead of computing each hidden state as a direct function of inputs and other hidden states, we commpmute it as a function of the LSTM cell's value (cell state) at that timestep. Each cell state is in turn functionally dependent on the previous cell state and any available or previous hidden states.

In other words, hidden states are computed from cell states, and cell states are computed from past or shallow hidden states.

### Cell state 
cell state at a specific timestep **t** is denoted $c_t$

c_t, as highlighted by the three arrows pointing towards it, has multiple potential functional dependencies. Four to be exact, though only a maximum of three can exist at once. These are:

- __The previous hidden state in time__: h_t-1. Again, if t = 1, then this won’t exist. If it does, this would be the first arrow pointing into the left side of c_t.
- __The previous cell state__: c_t-1. If t = 1, the dependency obviously won’t exist. This refers to the second arrow pointing into the left side of c_t.
- __Input at the current timestep__: x_t. There may very well be no input available, for example if we are at a hidden layer ℓ > 1. So this dependency doesn’t always exist. When it does, it’s the arrow pointing into the bottom of c_t.
- __The previous hidden state in depth__: h^(ℓ-1)_t. This applies for any hidden layer ℓ > 1. In such case, it would — like the input x_t — be the arrow pointing into the bottom.

Only three can exist at once because the last two are mutually exclusive.

From there, we pass information to the next cell state c_t+1 and compute h_t. As you can hopefully see, h_t then goes on to also influence c_t+1 

### Cell state conveyor belt example
Thing of the cell state as a conveyor belt. The reason why is because infomation cal flow through a cell super easily and its possible for info to speed past a single cell without bieng modified at all.

In vanilla RNNs, each hidden state takes all the information it has from before and fully transforms it by applying a function over it. LSTM cells instead take information and minor modification (like addition or multiplication)

__Regular RNN:__
$ h_{t-1} $ ---> $ f_W $ ---> $ h_t $ 

Vanilla RNNs look something like that above. And it’s why the vanishing gradient problem exists; during backprop, gradients cannot flow back past these transformations easily, because the tanh derivatives and weights chain up and multiply together and tend to zero. We then add up or average all these gradients that are basically zero, and we get zero.

__LSTM:__
$c_{t-1}$ ---> $ f_W (c_{t-1}) + c_{t-1} $ ---> $ c_t $

Sort of what an LSTM looks like. The previous timestep's cell state value flows through and instead of transforming the information, we tweak it by adding another vector to it. The added term is some function **fw** of previous information, but this is not the sme funtion as with the vanilla RNNs.

__Equation form:__
$$ c_t = c_{t-1} + f_w(c_{t-1}) $$

__Expanded form:__
$$ c_t = c_{1} + f_w(c_{1}) + f_w(c_{1}) +..+ f_w(c_{t-1})$$

With RNN: y = f1(f2(f3)

With LSTM: y = f1 + f2 + f3

LSTMS retain information that is useful and writes or forgets information that is not useful
$
\begin{bmatrix}
    a \\
    b \\
\end{bmatrix}
$
x
$
\begin{bmatrix}
    c \\
    d \\
\end{bmatrix}
$
=
$
\begin{bmatrix}
    ac \\
    bd \\
\end{bmatrix}
$

When a = 0, the information of c is lost. Values such as 0.5 can be used to reduce the importance of certain information.




### Cell state gating
Gating - allows what can flow in and out of LSTM cell.
"x" = reset or read
"+" = write

- f: forget gate. This is the “reset” tool that wipes out, diminishes, or retains information from the previous cell state. It’s the first interaction we make, and it’s multiplicative. That is, we multiply it with the cell state. The sigmoid function is used to compute the forget gate such that its values can be in the range 0 to 1. When a value is 1, we “remember” something, and when it is 0 we “forget”. We might choose to forget, for example, when see a period or some sort of end of sentence marker. This is counterintuitive… I guess it should really be called the “remember gate”!

- g: ?. This gate doesn’t really have a name, but it’s partly responsible for the “write” process. It stores a value between -1 and 1 that represents how much we want to add to the cell state by, and represents the input to the cell state. It’s computed with the tanh function. We apply a bounded function to it such that the cell state acts as a stable counter, and it also introduces more complexity. (And it works well.)

- i: input gate. This is the other gate responsible for the “write” process. It controls how much of g we “let in”, and is thus between 0 and 1, computed with sigmoid. It’s similar to the forget gate in this sense, in that it blocks input like the forget gate blocks the incoming cell state. We multiply i by g and add this to the cell state. Since i is in the range 0 to 1, and g is in the range -1 to 1, we add a value between -1 and 1 to the cell state. Intuitively, this sort of acts as decrementing or incrementing the counter.

- o: output gate. This is also passed through sigmoid, and is a number between 0 and 1 that modulates which aspects the hidden state can draw from the cell state. It enables the “read from memory” operation. It multiplies with the tanh of the cell state to compute the hidden state. So, I didn’t bring this up before, but the cell state leaks into a tanh before h_t is computed.

Overall, __f__ interacts with cell state through multiplication. __i__ interacts with __g__ through multiplication with multiplication. The result of this interacts with the cell state through addition. Lastly, the cell state leaks into a tanh. the result of of then reacts with __o__ through multiplication to compute $h_t$. $h_t$ then flows forward.

__Equation form:__
$$ c_t = (f x c_{t-1}) + (i x g)$$
$$ h_t = o x tanh(c_t) $$
